In [1]:
!pip install kaggle


In [2]:
import os
os.makedirs("/root/.kaggle", exist_ok=True)

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"regidor24","key":"c0a94ad325e7da67b7774933fadf51fc"}'}

In [4]:
!mv kaggle.json /root/.kaggle/

In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d domingosnhamussusa/earthquake-damage-in-nepal

import zipfile
zip_ref = zipfile.ZipFile('/content/earthquake-damage-in-nepal.zip', 'r')
zip_ref.extractall('/content/earthquake_data')
zip_ref.close()

print("Dataset ready!")


Dataset URL: https://www.kaggle.com/datasets/domingosnhamussusa/earthquake-damage-in-nepal
License(s): unknown
  0% 0.00/7.78M [00:00<?, ?B/s]
100% 7.78M/7.78M [00:00<00:00, 1.07GB/s]
Dataset ready!


In [7]:
import pandas as pd

df = pd.read_csv('/content/earthquake_data/NepalEarhquakeDamage2015.csv')
df.head()

,building_id,district_id,count_floors_pre_eq,count_floors_post_eq,age_building,plinth_area_sq_ft,height_ft_pre_eq,height_ft_post_eq,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,condition_post_eq,damage_grade
0,120101000011,12,1,1,9,288,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,Damaged-Used in risk,Grade 3
1,120101000021,12,1,1,15,364,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 5
2,120101000031,12,1,1,20,384,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 2
3,120101000041,12,1,1,20,312,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 2
4,120101000051,12,1,1,30,308,9,9,Flat,Other,Bamboo/Timber-Light roof,Mud,Not applicable,Not attached,Rectangular,Damaged-Repaired and used,Grade 1


In [8]:
columns_to_drop = [
    'building_id',
    'district_id',
    'count_floors_post_eq',
    'height_ft_post_eq',
    'condition_post_eq'
]

df_cleaned = df.drop(columns=columns_to_drop)
print(f"Columns dropped: {columns_to_drop}")
print("Shape after dropping columns:", df_cleaned.shape)

df_cleaned.dropna(inplace=True)
print("DataFrame after dropping rows with missing values:")
print(df_cleaned.head())
print("Shape after handling missing values:", df_cleaned.shape)
print("Missing values check after cleaning:", df_cleaned.isnull().sum().sum())

Columns dropped: ['building_id', 'district_id', 'count_floors_post_eq', 'height_ft_post_eq', 'condition_post_eq']
Shape after dropping columns: (762106, 12)
DataFrame after dropping rows with missing values:
   count_floors_pre_eq  age_building  plinth_area_sq_ft  height_ft_pre_eq  \
0                    1             9                288                 9   
1                    1            15                364                 9   
2                    1            20                384                 9   
3                    1            20                312                 9   
4                    1            30                308                 9   

  land_surface_condition foundation_type                 roof_type  \
0                   Flat           Other  Bamboo/Timber-Light roof   
1                   Flat           Other  Bamboo/Timber-Light roof   
2                   Flat           Other  Bamboo/Timber-Light roof   
3                   Flat           Other  Bamboo/

In [9]:

print(df.shape)
print(df.isnull().sum())

df = df.fillna(method='ffill').fillna(method='bfill')


(762106, 17)
building_id                0
district_id                0
count_floors_pre_eq        0
count_floors_post_eq       0
age_building               0
plinth_area_sq_ft          0
height_ft_pre_eq           0
height_ft_post_eq          0
land_surface_condition     0
foundation_type            0
roof_type                  0
ground_floor_type          0
other_floor_type           0
position                   1
plan_configuration         1
condition_post_eq          0
damage_grade              12
dtype: int64


/tmp/ipython-input-2578902962.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').fillna(method='bfill')


In [ ]:
!pip install category-encoders
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer


data = df.copy()
target_col = 'damage_grade'

# Extract target variable 'y' and drop it from 'data' (features)
y = data[target_col]
data = data.drop(columns=[target_col])

# Convert target labels to numerical format
le = LabelEncoder()
y = pd.Series(le.fit_transform(y), index=y.index)

if 'age' in data.columns:
    data['age_bucket'] = pd.cut(data['age'], bins=[-1,5,15,30,60,200], labels=['new','young','mid','old','very_old'])


cat_cols = data.select_dtypes(include=['object','category']).columns.tolist()

maybe_cat = ['foundation_type','land_surface_condition','roof_type','ground_floor_type','position']
for c in maybe_cat:
    if c in data.columns and data[c].nunique() < 30:
        cat_cols.append(c)

cat_cols = sorted(list(set(cat_cols)))
# The target_col is already removed from data, so no need to filter it out here
num_cols = [c for c in data.columns if c not in cat_cols]


high_card_threshold = 20
te_cols = [c for c in cat_cols if data[c].nunique() > high_card_threshold]
ohe_cols = [c for c in cat_cols if c not in te_cols]


num_imputer = SimpleImputer(strategy='median')
data[num_cols] = num_imputer.fit_transform(data[num_cols])


data = pd.get_dummies(data, columns=ohe_cols, drop_first=True)


if 'age' in data.columns and 'count_floors_pre_eq' in data.columns:
    data['age_x_floors'] = data['age'] * data['count_floors_pre_eq']


scaler = StandardScaler()
data[num_cols] = scaler.fit_transform(data[num_cols])


X = data
# y is already defined above

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# -----------------------------
# Identify columns
# -----------------------------
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object", "category"]).columns

# -----------------------------
# 1. Handle missing values
# -----------------------------
num_imputer = SimpleImputer(strategy="median")
X[num_cols] = num_imputer.fit_transform(X[num_cols])

if len(cat_cols) > 0:
    cat_imputer = SimpleImputer(strategy="most_frequent")
    X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

# -----------------------------
# 2. Handle outliers (IQR clipping)
# -----------------------------
for col in num_cols:
    Q1 = X[col].quantile(0.25)
    Q3 = X[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    X[col] = np.clip(X[col], lower, upper)

# -----------------------------
# 3. Feature scaling
# -----------------------------
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

print("Missing values handled, outliers capped, features scaled safely.")


In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


from category_encoders import TargetEncoder
te = TargetEncoder(cols=te_cols)
if te_cols:
    X_train = X_train.copy()
    X_test = X_test.copy()
    X_train[te_cols] = te.fit_transform(X_train[te_cols], y_train)
    X_test[te_cols] = te.transform(X_test[te_cols])


In [ ]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix


xgb_clf = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
xgb_clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])


lgb_clf = lgb.LGBMClassifier(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    colsample_bytree=0.8,
    subsample=0.8,
    random_state=42
)
lgb_clf.fit(X_train, y_train, eval_set=[(X_test, y_test)])


for name, model in [('XGBoost', xgb_clf), ('LightGBM', lgb_clf)]:
    preds = model.predict(X_test)
    print(f'=== {name} ===')
    print('Accuracy:', accuracy_score(y_test, preds))
    print(classification_report(y_test, preds))

In [ ]:
!pip install optuna

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import joblib

# Save the LightGBM model
joblib.dump(lgb_clf, 'lightgbm_model.pkl')

print("LightGBM model saved as 'lightgbm_model.pkl'")

In [ ]:
import joblib
loaded_model = joblib.load('lightgbm_model.pkl')

To load the model later, you can use the following code:
```python
import joblib
loaded_model = joblib.load('lightgbm_model.pkl')
```

In [ ]:
import joblib
joblib.dump(lgb_clf, 'lgb_model.joblib')
joblib.dump(xgb_clf, 'xgb_model.joblib')



In [ ]:
import joblib


final_training_features = X.columns.tolist()


joblib.dump(final_training_features, "features.joblib")

print(f"Saved {len(final_training_features)} features")


In [ ]:
final_model_features = joblib.load("features.joblib")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import joblib

joblib.dump(lgb_clf, 'lgb_model.joblib')
joblib.dump(xgb_clf, 'xgb_model.joblib')

joblib.dump(scaler, 'scaler.joblib')
joblib.dump(num_imputer, 'num_imputer.joblib')
joblib.dump(te, 'target_encoder.joblib')
joblib.dump(le, 'label_encoder.joblib')
joblib.dump(num_cols, 'num_cols.joblib')
joblib.dump(ohe_cols, 'ohe_cols.joblib')
joblib.dump(te_cols, 'te_cols.joblib')
joblib.dump(X_train.columns.tolist(), 'final_training_columns.joblib')

print("Models & preprocessors saved")

In [ ]:
def preprocess_input(df, training=False):
    import joblib
    import pandas as pd

    scaler = joblib.load('scaler.joblib')
    num_imputer = joblib.load('num_imputer.joblib')
    te = joblib.load('target_encoder.joblib')
    num_cols = joblib.load('num_cols.joblib')
    ohe_cols = joblib.load('ohe_cols.joblib')
    te_cols = joblib.load('te_cols.joblib')

    df = df.copy()

    if 'age' in df.columns:
        df['age_bucket'] = pd.cut(
            df['age'],
            bins=[-1,5,15,30,60,200],
            labels=['new','young','mid','old','very_old']
        )

    df[num_cols] = num_imputer.transform(df[num_cols])

    df = pd.get_dummies(df, columns=ohe_cols, drop_first=True)

    for col in te_cols:
        if col not in df.columns:
            df[col] = "Unknown"

    df[te_cols] = te.transform(df[te_cols])

    for col in scaler.feature_names_in_:
        if col not in df.columns:
            df[col] = 0

    df = df[scaler.feature_names_in_]
    df[num_cols] = scaler.transform(df[num_cols])

    return df


In [ ]:
def predict_damage(input_df):
    import joblib

    model = joblib.load('lgb_model.joblib')
    label_encoder = joblib.load('label_encoder.joblib')

    processed = preprocess_input(input_df)
    pred = model.predict(processed)
    prob = model.predict_proba(processed)

    damage_label = label_encoder.inverse_transform(pred)[0]
    confidence = prob.max()

    return damage_label, confidence


In [ ]:
import pandas as pd

def statistical_summary(df):
    summary = pd.DataFrame({
        "Missing Values": df.isnull().sum(),
        "Missing %": (df.isnull().mean() * 100).round(2),
        "Min": df.min(numeric_only=True),
        "Max": df.max(numeric_only=True),
        "Median": df.median(numeric_only=True),
        "Mean": df.mean(numeric_only=True),
        "Std Dev": df.std(numeric_only=True)
    })
    return summary

summary_table = statistical_summary(df_cleaned)
summary_table


In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import lightgbm as lgb
import xgboost as xgb

# Load dataset
df = pd.read_csv('/content/earthquake_data/NepalEarhquakeDamage2015.csv')

# Final UI-safe features
features = [
    'age_building',
    'count_floors_pre_eq',
    'foundation_type',
    'roof_type',
    'ground_floor_type',
    'position',
    'land_surface_condition'
]

target = 'damage_grade'

df = df[features + [target]]
df.dropna(inplace=True)

# Encode target using LabelEncoder
le_ui = LabelEncoder() # Create a new LabelEncoder for the UI model
df[target] = le_ui.fit_transform(df[target])

# One-hot encode categorical features
X = pd.get_dummies(df[features])
y = df[target]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

num_cols = ['age_building', 'count_floors_pre_eq']

# Impute
num_imputer = SimpleImputer(strategy='median')
X_train[num_cols] = num_imputer.fit_transform(X_train[num_cols])
X_test[num_cols] = num_imputer.transform(X_test[num_cols])

# Scale
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

# Train LightGBM model
lgb_model = lgb.LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)
lgb_model.fit(X_train, y_train)

# Train XGBoost model with the same reduced feature set
xgb_model = xgb.XGBClassifier(
    n_estimators=300, # Reduced for faster training
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    use_label_encoder=False, # Suppress warning
    eval_metric='mlogloss'
)
xgb_model.fit(X_train, y_train)

# SAVE EVERYTHING
joblib.dump(lgb_model, 'lgb_model.joblib')
joblib.dump(xgb_model, 'xgb_model.joblib') # Save the new XGBoost model
joblib.dump(X.columns.tolist(), 'features.joblib')
joblib.dump(num_imputer, 'num_imputer.joblib')
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(le_ui, 'label_encoder.joblib') # Save the LabelEncoder used here

print("Models, preprocessors, and features.joblib created successfully with UI-safe features")


In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd
from prediction import predict_damage

st.set_page_config(
    page_title="Earthquake Damage Predictor",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ---- Constants ----
MODEL_ACCURACY = 0.76  # 76%

def confidence_to_certainty(conf):
    if conf < 0.35:
        return "Low"
    elif conf < 0.50:
        return "Moderate"
    elif conf < 0.65:
        return "High"
    else:
        return "Very High"

# ---- Sidebar ----
st.sidebar.header("Input Parameters")
st.sidebar.write("Provide the building details for prediction.")

with st.sidebar.form("input_form"):
    st.subheader("Structural Features")
    age = st.number_input("Building Age (years)", 0, 200, 20)
    floors = st.number_input("Number of Floors", 1, 10, 2)

    st.subheader("Construction Details")
    foundation = st.selectbox("Foundation Type", ['mud', 'cement', 'other'])
    roof = st.selectbox("Roof Type", ['bamboo', 'metal', 'concrete', 'other'])
    ground = st.selectbox("Ground Floor Type", ['mud', 'cement', 'other'])

    st.subheader("Position & Land")
    position = st.selectbox("Building Position", ['attached', 'not_attached'])
    land_surface = st.selectbox("Land Surface Condition", ['flat', 'slope', 'other'])

    submit = st.form_submit_button("Predict")

# ---- Main Panel ----
st.title("Earthquake Building Damage Prediction")
st.write("Predict potential building damage based on structural characteristics.")

if submit:
    input_df = pd.DataFrame([{
        'age': age,
        'count_floors_pre_eq': floors,
        'foundation_type': foundation,
        'roof_type': roof,
        'ground_floor_type': ground,
        'position': position,
        'land_surface_condition': land_surface
    }])

    label, confidence = predict_damage(input_df)
    certainty = confidence_to_certainty(confidence)

    st.markdown("---")
    st.header("Prediction Result")

    # Display main prediction
    st.success(f"Predicted Damage Level: **{label.upper()}**")

    # Additional information in columns for clarity
    col1, col2 = st.columns(2)
    col1.write(f"**Model Certainty:** {certainty}")
    col2.write(f"**Overall Model Accuracy:** {int(MODEL_ACCURACY * 100)}%")

    # Optional technical details
    with st.expander("See Technical Details"):
        st.write(f"Raw prediction confidence: **{confidence:.2%}**")
        st.caption(
            "Note: In multi-class damage prediction, moderate confidence is expected due to overlapping damage categories."
        )

# ---- Footer ----
st.markdown("---")
st.caption("All predictions are based on model estimation. Use as a guide, not a definitive assessment.")


In [ ]:
%%writefile prediction.py

import joblib
import pandas as pd

def preprocess_input(df_raw, training=False):
    df = df_raw.copy()

    # Load all preprocessors and column lists
    scaler = joblib.load('scaler.joblib')
    num_imputer = joblib.load('num_imputer.joblib')

    # The features.joblib saved in nOkF82Xv2mKV is X.columns.tolist() which are the FINAL OHE features for the UI-safe model.
    final_model_features = joblib.load('features.joblib') # Renaming this for clarity

    # --- Manual definition of numerical columns for the UI-safe model ---
    # These are the columns the num_imputer and scaler were actually fitted on in nOkF82Xv2mKV
    num_cols_ui_safe = ['age_building', 'count_floors_pre_eq']

    # --- Step 1: Align input column names ---
    if 'age' in df.columns:
        df.rename(columns={'age': 'age_building'}, inplace=True)

    # --- Step 2: Extract and preprocess numerical features ---
    # Create a temporary DataFrame for numerical features from the UI input
    # Initialize with 0s and then fill if available in df
    df_numerical_subset = pd.DataFrame(0, index=df.index, columns=num_cols_ui_safe)
    for col in num_cols_ui_safe:
        if col in df.columns:
            df_numerical_subset[col] = df[col]

    # Apply numerical imputation and scaling to this subset
    df_numerical_subset[num_cols_ui_safe] = num_imputer.transform(df_numerical_subset[num_cols_ui_safe])
    df_numerical_subset[num_cols_ui_safe] = scaler.transform(df_numerical_subset[num_cols_ui_safe])

    # --- Step 3: Handle Categorical Features (One-Hot Encoding) ---
    # Identify categorical columns from the raw input that are NOT numerical
    # This assumes `df` contains the raw categorical inputs like 'foundation_type', 'roof_type' etc.
    categorical_raw_cols_from_ui = [col for col in df.columns if col not in num_cols_ui_safe and df[col].dtype == 'object']
    df_processed_categorical = pd.get_dummies(df[categorical_raw_cols_from_ui], drop_first=True)

    # --- Step 4: Combine numerical and categorical features ---
    # Combine the processed numerical and categorical features
    processed_features = pd.concat([df_numerical_subset, df_processed_categorical], axis=1)

    # --- Step 5: Final column alignment with the model's training features ---
    # Create a final DataFrame with all columns from final_model_features, filled with 0 if missing
    final_df = pd.DataFrame(0, index=processed_features.index, columns=final_model_features)
    for col in final_model_features:
        if col in processed_features.columns:
            final_df[col] = processed_features[col]

    return final_df

def predict_damage(input_df):
    model = joblib.load('lgb_model.joblib')
    label_encoder = joblib.load('label_encoder.joblib')

    processed = preprocess_input(input_df)
    pred = model.predict(processed)
    prob = model.predict_proba(processed)

    damage_label = label_encoder.inverse_transform(pred)[0]
    confidence = prob.max()

    return damage_label, confidence

In [ ]:
from prediction import preprocess_input

In [ ]:

!pip -q install pyngrok streamlit > /dev/null 2>&1

from pyngrok import ngrok
import getpass
import subprocess


auth_token = getpass.getpass("Enter your ngrok authtoken (or press Enter if set below): ")
if not auth_token.strip():

    auth_token = "36VH5tP1L1rSZvhq3LXCr6rLauL_4xBtyvUjAj4BRhg8zWTjV"

ngrok.set_auth_token(auth_token)


public_url = ngrok.connect(8501)
print("Public URL:", public_url)


subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"])

#### Data Distribution: Building Ages (Histogram)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.histplot(df['age_building'], bins=30, kde=True)
plt.title('Distribution of Building Ages')
plt.xlabel('Building Age (years)')
plt.ylabel('Number of Buildings')
plt.grid(axis='y', alpha=0.75)
plt.show()

#### Data Distribution: Foundation Types (Bar Chart)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.countplot(y=df['foundation_type'], order=df['foundation_type'].value_counts().index)
plt.title('Distribution of Foundation Types')
plt.xlabel('Number of Buildings')
plt.ylabel('Foundation Type')
plt.grid(axis='x', alpha=0.75)
plt.show()

#### Relationship: Building Age vs. Number of Floors (Scatter Plot)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.scatterplot(x=df['age_building'], y=df['count_floors_pre_eq'], alpha=0.6)
plt.title('Relationship between Building Age and Number of Floors')
plt.xlabel('Building Age (years)')
plt.ylabel('Number of Floors')
plt.grid(True, alpha=0.75)
plt.show()

#### Model Performance: Confusion Matrix (LightGBM)

This confusion matrix will show you how many instances of each true damage grade were predicted as each possible damage grade. The diagonal elements represent correct predictions, while off-diagonal elements show misclassifications.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import joblib


lgb_clf = joblib.load('lgb_model.joblib')
le_ui = joblib.load('label_encoder.joblib')


y_pred_lgbm = lgb_clf.predict(X_test)


cm = confusion_matrix(y_test, y_pred_lgbm)


damage_labels = le_ui.inverse_transform(sorted(y_test.unique()))


plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=damage_labels, yticklabels=damage_labels)
plt.title('Confusion Matrix for LightGBM Model')
plt.xlabel('Predicted Damage Grade')
plt.ylabel('True Damage Grade')
plt.show()

#### Model Performance: Confusion Matrix (XGBoost)

This confusion matrix shows how many instances of each true damage grade were predicted as each possible damage grade for the XGBoost model. The diagonal elements represent correct predictions, while off-diagonal elements show misclassifications.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import joblib

# Load the trained XGBoost model and label encoder
xgb_clf = joblib.load('xgb_model.joblib')
le_ui = joblib.load('label_encoder.joblib')

# Make predictions on the test set
y_pred_xgb = xgb_clf.predict(X_test)

# Calculate the confusion matrix
cm_xgb = confusion_matrix(y_test, y_pred_xgb)

# Get damage labels for plotting
damage_labels = le_ui.inverse_transform(sorted(y_test.unique()))

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues',
            xticklabels=damage_labels, yticklabels=damage_labels)
plt.title('Confusion Matrix for XGBoost Model')
plt.xlabel('Predicted Damage Grade')
plt.ylabel('True Damage Grade')
plt.show()


## Expected Outputs of the Streamlit Application

Upon successful submission of building parameters, the Streamlit application is designed to provide clear and actionable insights into the predicted earthquake damage level. The output section will dynamically update to display the model's prediction along with confidence measures.

### Output Structure:

1.  **Prediction Result Header**: A prominent header indicating the section for prediction results.

2.  **Predicted Damage Level**: The primary output, displayed prominently, will be the predicted damage grade for the input building. This will be shown as:
    *   **`Predicted Damage Level: [GRADE]`** (e.g., `Predicted Damage Level: GRADE 3`)
    
3.  **Confidence and Accuracy Metrics**: Presented in a columnar layout for readability, these metrics provide context to the prediction:
    *   **Model Certainty**: An qualitative assessment of the model's confidence in its prediction, categorized as:
        *   `Low` (confidence < 35%)
        *   `Moderate` (35% <= confidence < 50%)
        *   `High` (50% <= confidence < 65%)
        *   `Very High` (confidence >= 65%)
    *   **Overall Model Accuracy**: A static display of the general accuracy of the underlying machine learning model, typically shown as a percentage (e.g., `Overall Model Accuracy: 76%`).

4.  **Technical Details (Expandable)**: An expandable section (`st.expander`) provides deeper insights for users interested in the model's raw output:
    *   **Raw prediction confidence**: The exact probability score (as a percentage) that the model assigns to the predicted damage grade (e.g., `Raw prediction confidence: 72.50%`).
    *   **Note**: A brief explanation about why moderate confidence is expected in multi-class damage prediction due to overlapping categories.

### Example Output (Illustrative):

```
---
### Prediction Result

✅ Predicted Damage Level: **GRADE 3**

| Model Certainty    | Overall Model Accuracy |
|--------------------|------------------------|
| **Very High**      | **76%**                |

<details>
<summary>See Technical Details</summary>

Raw prediction confidence: **78.23%**
Note: In multi-class damage prediction, moderate confidence is expected due to overlapping damage categories.

</details>

---
All predictions are based on model estimation. Use as a guide, not a definitive assessment.
```

This structure ensures that users receive both an easy-to-understand summary and detailed information about the prediction.

## Model Evaluation Metrics

This section details the performance of the trained classification models (XGBoost and LightGBM) on the test dataset. The primary goal is to assess their ability to accurately predict the damage grade of buildings after an earthquake.

### Metrics Used:

*   **Accuracy**: The proportion of correctly classified instances.
*   **Precision**: The ability of the classifier not to label as positive a sample that is negative (i.e., fewer false positives).
*   **Recall**: The ability of the classifier to find all the positive samples (i.e., fewer false negatives).
*   **F1-Score**: The harmonic mean of precision and recall, providing a single metric that balances both.
*   **Support**: The number of actual occurrences of the class in the specified dataset.
*   **Macro Average**: Calculates metrics for each label, and finds their unweighted mean.
*   **Weighted Average**: Calculates metrics for each label, and finds their average weighted by support.

### Results:

#### XGBoost Classifier

*   **Overall Accuracy**: 0.7552

| Damage Grade | Precision | Recall | F1-Score | Support |
|--------------|-----------|--------|----------|---------|
| Grade 0      | 0.93      | 0.85   | 0.89     | 15763   |
| Grade 1      | 0.53      | 0.44   | 0.48     | 17452   |
| Grade 2      | 0.49      | 0.41   | 0.45     | 27284   |
| Grade 3      | 0.63      | 0.82   | 0.71     | 36769   |
| Grade 4      | 1.00      | 0.95   | 0.98     | 55154   |
| **Macro Avg**| 0.72      | 0.70   | 0.70     | 152422  |
| **Weighted Avg**| 0.76      | 0.76   | 0.75     | 152422  |

#### LightGBM Classifier

*   **Overall Accuracy**: 0.7655

| Damage Grade | Precision | Recall | F1-Score | Support |
|--------------|-----------|--------|----------|---------|
| Grade 0      | 0.92      | 0.87   | 0.89     | 15763   |
| Grade 1      | 0.54      | 0.46   | 0.50     | 17452   |
| Grade 2      | 0.51      | 0.45   | 0.48     | 27284   |
| Grade 3      | 0.65      | 0.81   | 0.72     | 36769   |
| Grade 4      | 1.00      | 0.96   | 0.98     | 55154   |
| **Macro Avg**| 0.73      | 0.71   | 0.71     | 152422  |
| **Weighted Avg**| 0.77      | 0.77   | 0.76     | 152422  |

### Summary and Comparison:

Both models achieved good overall accuracy. The LightGBM classifier showed slightly higher overall accuracy (0.7655) compared to XGBoost (0.7552) on this dataset. Both models performed exceptionally well in identifying 'Grade 4' damage (severe damage) with high precision and recall. However, 'Grade 1' and 'Grade 2' damage predictions show lower precision and recall, indicating more difficulty in distinguishing between these categories. The weighted average metrics, which account for class imbalance, are generally higher than the macro averages, reflecting the models' stronger performance on the more prevalent damage grades.

For future improvements, focusing on the classification of 'Grade 1' and 'Grade 2' could significantly enhance the model's overall F1-score and its utility for nuanced damage assessment.

Median Imputation: "I used median imputation because it's good for handling missing values in numerical data without being heavily affected by extreme values (outliers), which could happen if I used the mean."

StandardScaler: "I used StandardScaler to make sure all my numerical features have a similar scale. This is important because many machine learning models, like the ones I used, can get confused if one feature has very large numbers and another has very small numbers."

IQR Clipping (Outlier Handling): "I used IQR (Interquartile Range) clipping to handle outliers, which are unusual data points. This method helps to limit very high or very low values, making sure they don't unfairly influence the model,
which might otherwise learn from noise instead of real patterns."

LightGBM vs. XGBoost: "LightGBM performed slightly better than XGBoost in my tests. This could be because LightGBM is generally faster and often performs well on large datasets, and it might have found slightly better patterns in this particular dataset." (You can also mention its 'leaf-wise' growth strategy as a technical detail if you feel comfortable).




By analyzing feature importance, we can see which building characteristics, such as the age_building or foundation_type, had the biggest impact on predicting damage. This helps us understand what factors are most critical for a building's resilience to earthquakes